<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js024_sampleLDA2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js024_sampleLDA2D / def exec_html_js() ... exec me first

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js024_sampleLDA2D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js024_sampleLDA2D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.05.13 created, last updated on 2018.11.16
//    ver 0.0.1  2019.01.15 v1, last updated on 2021.05.13
//    ver 0.0.2  2021.10.30 v2, last updated on 2021.10.30
//    ver 0.0.3  2023.03.08 v3, last updated on 2023.08.20
//
// -------------------- RS-DFT - local density approximation 2D
//
// - real space density functional theory - local density approximation
// - solve Kohn-Sham equation - successive approximation
// - Vxc : LDA(local density approximation)
//         J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
//
//  many electron system --> one electron approximation
//    one electron ,  others -> consider as effective potential Veff
//    Kohn-Sham equation {(-d^2/dr^2)+Veff(r)} |i> = e_i |i>,  Veff(r)=Vext+VH+Vx+Vc
//      Veff: one electron feels effective potential
//      Vext: external Potential
//      VH  : Hartree potential (electron charge -> electro-static potential)
//      Vx,Vc :electron exchange and correlation potential (LDA:Perdew-Zunger)
//    successive approximation
//      solve Kohn-Sham -> electron state,occupation -> electron charge density -> Veff
//
//  procedure
//   (1) given: trial |i>, occupation(i)
//
//   (2) set electron density rho
//       rho <-- |i>, occupation[(i), mixing rho(iter-1)
//
//   (3) set effective potential
//        Veff = Vext + VH + Vx + Vc
//        VH <-- rho (Poisson eq. ,SOR iteration)
//        Vx,Vc <-- rho (LDA:Perdew-Zunger)
//
//   (4) solve Kohn-Sham equation (successive approximation)
//        |i> steepest descent method: |i(next)> = |i> - dump{H-E}|i>
//        E(i) <-- <i|H|i>
//        {|0>,..,|i>,..,|N>} orthogonallization : Gram-Schmidt
//
//   (5) sort state
//        sort orbit by E(i)
//
//   (6) set occupation
//        occupation(i) <-- E(i)
//
//   goto (2)
//
// --------------------
*/

const sampleLDA2D = (function(){ // ====================  sampleLDA2D Module  ====================

	// au: atomic unit hBar=1,e=1,me=1,a0=1
	const g_auLength = 5.29177211e-11;			// (m) 1(au) = auLength (m)
	const g_auTime = 2.418884326e-17;			// (s) 1(au) = auTime (s)
	const g_auEnergy = 4.35974465e-18;			// (J) 1(au) = auEnergy (J)
	const g_au2eV = 27.211386;					// (eV) 1(au) = 27.211386 (eV)
	const g_nMax = 128;							// maximum of NNx
	const g_stMax = 20;							// maximum number of state

	let g_iterCount = 0;						// sd iteration count
	let g_numberOfElectron = 4;					// number of electron
	let g_numberOfOrbit = 10;					// number of orbit
	let g_NNx = 64;								// max number of sdState(,NNx,NNy)
	let g_NNy = 64;								//
	let g_dx = 1.0/4.0;							// (au) x-division
	let g_dy = 1.0/4.0;							// (au) y-division
	let g_lz = 16.0;							// (au) dv=dx*dy*lz (to use LDA rho -> Vxc)
	let g_energyMem = 0.0;						// temporal memory of energy
	let g_iterationError = 1.0;					// energy difference for iteration n and n+1
	let g_dampingFactor = 0.01;					// damping factor of steepest descent method
	let g_mixing = 0.5;							// charge mixing in setRho()
	let g_broadening = 0.001;					// (au) level broadening IN setOccupation

	const g_sdEnergy = dim1( g_stMax );			// sdEnergy[ist]    electron orbit energy
	const g_sdState = dim3( g_stMax, g_nMax, g_nMax ); // sdState[ist][i][j] electron orbit
	const g_occupation = dim1( g_stMax );		// occupation[ist]  occupation of orbit  0.0 ... 2.0
	const g_wrk = dim2( g_nMax, g_nMax );		// wrk[i][j]   orbit work space in steepestDescent()
	const g_vv = dim2( g_nMax, g_nMax );		// vv[i][j]    effective potential
	const g_vvext = dim2( g_nMax, g_nMax );		// vvext[i][j] external potential
	const g_vvh = dim2( g_nMax, g_nMax );		// vvh[i][j]   Hartree potential
	const g_vvx = dim2( g_nMax, g_nMax );		// vvx[i][j]   exchange potential
	const g_vvc = dim2( g_nMax, g_nMax );		// vvc[i][j]   correlation potential
	const g_rho = dim2( g_nMax, g_nMax );		// rho[i][j]   electron charge density
	const g_cloud = dimInt2( g_nMax, g_nMax );	// if cloud[i][j]>0, plot cloud point

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dim2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Float64Array( nj );
		}
		return a;
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}


	// --------------------  set initial condition  --------------------

	// public
	function setInitialCondition( stateMax, vIndex, nElectron ) {
		g_iterCount = 0;
		setInitialState(stateMax);
		setExternalPotential(vIndex);
		setNumberOfElectron( nElectron );
	}

	function setInitialState(stateMax) {
		const nnx=g_NNx, nny=g_NNy;
		for (let ist=0; ist<stateMax; ist++) {
			for (let j=0; j<nny; j++) {
				g_sdState[ist][0][j] = 0.0;
			}
			for (let i=1; i<nnx-1; i++) {
				g_sdState[ist][i][0] = 0.0;
				for (let j=1; j<nny-1; j++) {
					g_sdState[ist][i][j] = Math.random()-0.5;
				}
				g_sdState[ist][i][nny-1] = 0.0;
			}
			for (let j=0; j<nny; j++) {
				g_sdState[ist][nnx-1][j] = 0.0;
			}
			normalizeState(ist);
		}
	}

	function setExternalPotential(vIndex) {
		const nnx=g_NNx, nny=g_NNy;
		const x0 = 0.5*nnx*g_dx, y0 = 0.5*nny*g_dy;
		for (let i=0; i<nnx; i++) {
			const x = i*g_dx;
			for (let j=0; j<nny; j++) {
				const y = j*g_dy;
				const r2 = ((x-x0)*(x-x0)+(y-y0)*(y-y0));
				if (vIndex==0) { // 0:parabolic
					g_vvext[i][j] = Math.min(0.5*r2, 20.0);
				} else if (vIndex==1) { // 1:well
					g_vvext[i][j] = (r2<9.0) ? 0.0 : 20.0;
				} else if (vIndex==2) { // 1:well
					g_vvext[i][j] = (r2<16.0) ? -1.0 : 0.0;
				} else if (vIndex==3) { // 1:well
					g_vvext[i][j] = (r2>=16.0 && r2<36.0 ) ? 0.0 : 20.0;
				}
			}
		}
	}

	// public
	function setNumberOfElectron( ne ) {
		g_numberOfElectron = ne;
		g_iterCount = 0;
	}

	function setMixing( mix ) {
		g_mixing = mix;
	}

	function setBroadening( broadening ) {
		g_broadening = broadening;
	}


	// --------------------  iterate LDA  --------------------

	// public
	function iterateLDA( stateMax, iterMax ) {
		g_numberOfOrbit = stateMax;
		const errorDecisionOrbit = Math.floor((g_numberOfElectron-1)/2);
		setElectronDensity();
		setEffectivePotential();
		solveKohnSham(g_numberOfOrbit,iterMax);
		sortState(g_numberOfOrbit);
		setOccupation(g_numberOfOrbit,g_numberOfElectron);
		g_iterationError = g_sdEnergy[errorDecisionOrbit] - g_energyMem;
		g_energyMem = g_sdEnergy[errorDecisionOrbit];
	}

	//--- (2) set electron density rho -from sdState[], occupation[]

	function setElectronDensity() {
		const nnx=g_NNx, nny=g_NNy;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_rho[i][j] *= (1.0-g_mixing);
				for (let ie=0; ie<g_numberOfOrbit; ie++) {
					if (g_occupation[ie]<=0.0) continue;
					g_rho[i][j] += g_mixing*g_occupation[ie]*(g_sdState[ie][i][j]*g_sdState[ie][i][j])/g_lz;
				}
			}
		}
	}

	//--- (3) set effective potential -from electron density

	function setEffectivePotential() {
		const nnx=g_NNx, nny=g_NNy;
		poisson(20); // setVh
		setVxc();
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_vv[i][j] = g_vvext[i][j]+g_vvh[i][j]+g_vvx[i][j]+g_vvc[i][j];
			}
		}
	}

	function poisson(iterMax) { // solve (d^2/dr^2) V = 4 pai rho
		const nnx=g_NNx, nny=g_NNy, h2 = 4.0*Math.PI*g_dx*g_dx;
		const w = (1.0/4.0)*1.8; // 1/4 * SOR omega(1.0<omega<2.0)
		for (let iter=0; iter<iterMax; iter++) {
			for (let i=1; i<nnx-1; i++) {
				for (let j=1; j<nny-1; j++) {
					g_vvh[i][j] += w*(g_vvh[i+1][j]+g_vvh[i-1][j]+g_vvh[i][j+1]+g_vvh[i][j-1]-4.0*g_vvh[i][j]
										+h2*g_rho[i][j]);
				}
			}
			for (let j=0; j<nny; j++) {
				g_vvh[0][j] = 0.0;
				g_vvh[nnx-1][j] = 0.0;
			}
			for (let i=0; i<nnx; i++) {
				g_vvh[i][0] = 0.0;
				g_vvh[i][nny-1] = 0.0;
			}
		}
	}

	// LDA :  J. P. Perdew and A. Zunger; Phys. Rev., B23, 5048 (1981)
	function setVxc() {
		const nnx=g_NNx, nny=g_NNy;
		const c1 = -0.984745022;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const rh = g_rho[i][j];
				const rh3 = Math.pow(rh,0.33333333);
				g_vvx[i][j] = c1*rh3;
				const rs = 0.6204/(rh3+1.0e-20);
				if (rs>=1.0) {
					const sqrtrs = Math.sqrt(rs);
					const ec = -0.1423/(1.0+1.0529*sqrtrs+0.3334*rs);
					g_vvc[i][j] = ec*(1.0+1.22838*sqrtrs+0.4445*rs)/(1.0+1.0529*sqrtrs+0.3334*rs);
				} else {
					g_vvc[i][j] = -0.05837-0.0084*rs +(0.0311+0.00133*rs)*Math.log(rs);
				}
			}
		}
	}

	function eeCorrelation(rh) {
		let ec;
		const r = 0.6204/(Math.pow(rh,0.33333333)+1.0e-20);
		if (r>=1.0) {
			ec = -0.1423/(1.0+1.0529*Math.sqrt(r)+0.3334*r);
		} else {
			ec = -0.0480-0.0116*r+(0.0311+0.0020*r)*Math.log(r);
		}
		return ec;
	}

	//--- (4) solve Kohn-Sham equation

	function solveKohnSham(stateMax, iterMax) {
		for (let i=0; i<iterMax; i++) {
			for (let ist=0; ist<stateMax; ist++) {
				g_sdEnergy[ist] = steepestDescent(ist, g_dampingFactor);
			}
			GramSchmidt(stateMax);
			g_iterCount += 1;
		}
	}

	function steepestDescent(ist,damp) {
		const nnx=g_NNx, nny=g_NNy, h2=2*g_dx*g_dx, p = g_sdState[ist];
		const ei = energyOfState(ist);
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_wrk[i][j] = (4*p[i][j]-p[i+1][j]-p[i-1][j]-p[i][j+1]-p[i][j-1])/h2 + (g_vv[i][j]-ei)*p[i][j];
			}
		}
		for (let i=0; i<nnx-1; i++) {
			for (let j=0; j<nny-1; j++) {
				g_sdState[ist][i][j] -= damp*g_wrk[i][j];
			}
		}
		normalizeState(ist);
		return ei;
	}

	function energyOfState(ist) {
		const nnx=g_NNx, nny=g_NNy, h2=2*g_dx*g_dx, p=g_sdState[ist];
		let s=0.0, sn=0.0
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				s += p[i][j]*( (4*p[i][j]-p[i+1][j]-p[i-1][j]-p[i][j+1]-p[i][j-1])/h2 +　g_vv[i][j]*p[i][j]);
				sn += p[i][j]*p[i][j];
			}
		}
		return s/sn;
	}

	function GramSchmidt(stateMax) {
		const nnx=g_NNx, nny=g_NNy;
		normalizeState(0);
		for (let istate=1; istate<stateMax; istate++) {
			for (let ist=0; ist<istate; ist++) {
				const s = innerProduct(ist,istate);
				for (let i=1; i<nnx-1; i++) {
					for (let j=1; j<nny-1; j++) {
						g_sdState[istate][i][j] -= s*g_sdState[ist][i][j];
					}
				}
			}
			normalizeState(istate);
		}
	}

	//--- (5) sort state

	function sortState(stateMax) {
		const nnx=g_NNx, nny=g_NNy;

		for (let ist=stateMax-2; ist>=0; ist--) {
			if (g_sdEnergy[ist]>g_sdEnergy[ist+1]+0.00001) {
				for (let i=0; i<nnx; i++) {
					for (let j=0; j<nny; j++) {
						const w = g_sdState[ist][i][j];
						g_sdState[ist][i][j] = g_sdState[ist+1][i][j];
						g_sdState[ist+1][i][j] = w;
					}
				}
				const w = g_sdEnergy[ist];
				g_sdEnergy[ist] = g_sdEnergy[ist+1];
				g_sdEnergy[ist+1] = w;
			}
		}
	}

	//--- (6) set occupation

	function setOccupation(maxState, nElectron) {
		let eUpper = g_sdEnergy[maxState-1]+1.0;
		let eLower = g_sdEnergy[0]-1.0;
		for (let i=0; i<maxState; i++) {
			if(g_sdEnergy[i]>eUpper) eUpper = g_sdEnergy[i];
			if(g_sdEnergy[i]<eLower) eLower = g_sdEnergy[i];
		}
		while (eUpper-eLower>1.0e-12) {
			const eFermi = (eUpper+eLower)/2.0;
			const ntrial = trialOcc(maxState, eFermi);
			if (ntrial<nElectron) {
				eLower = eFermi;
			} else {
				eUpper = eFermi;
			}
		}
		const eFermi = (eUpper+eLower)/2.0;
		for (let i=0; i<maxState; i++) {
			g_occupation[i] = 2.0*FermiDirac(g_sdEnergy[i], eFermi);
			if (g_occupation[i]<0.0001) g_occupation[i] = 0.0;
			if (2.0-g_occupation[i]<0.0001) g_occupation[i] = 2.0;
		}
	}

	function trialOcc(maxState,eFermi) {
		let s = 0.0;
		for (let i=0;i<maxState;i++) {
			s += 2.0*FermiDirac(g_sdEnergy[i], eFermi);
		}
		return s;
	}

	function FermiDirac(ee, ef) {
		const et = g_broadening; //levelWidth
		return ( 1.0/(Math.exp((ee-ef)/et)+1.0) );
	}


	// --------------------  utility  --------------------

	function innerProduct(ist,jst) {
		const nnx=g_NNx, nny=g_NNy;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				s += g_sdState[ist][i][j]*g_sdState[jst][i][j];
			}
		}
		return s*g_dx*g_dy;
	}

	function normalizeState(ist) {
		const nnx=g_NNx, nny=g_NNy;
		let s = 0.0;
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				s += g_sdState[ist][i][j]*g_sdState[ist][i][j];
			}
		}
		const a = Math.sqrt(1.0/(s*g_dx*g_dy));
		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				g_sdState[ist][i][j] = a*g_sdState[ist][i][j];
			}
		}
	}

	function totalEnergy() {
		const nnx=g_NNx, nny=g_NNy;

		let sei = 0.0;
		for (let i=0; i<g_numberOfOrbit; i++) {
			sei += g_occupation[i]*g_sdEnergy[i];
		}
		let s = 0.0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				s += (-0.5*g_vvh[i][j]-0.25*g_vvx[i][j]+eeCorrelation(g_rho[i][j])-g_vvc[i][j])*g_rho[i][j];
			}
		}
		s = s*g_dx*g_dy;
		return sei+s;
	}


	// --------------------  cloud

	const g_srnd = [];

	function setCloud( ist ) {
		const nnx=g_NNx,nny=g_NNy;
		set_srnd();
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
			}
		}
		let s = 0, ip = 0;
		for (let i=1; i<nnx-1; i++) {
			for (let j=1; j<nny-1; j++) {
				g_cloud[i][j] = 0;
				const pij = g_sdState[ist][i][j];
				s += pij*pij*g_dx*g_dy;
				while (s>g_srnd[ip] && ip<1000) {
					g_cloud[i][j] += 1;
					ip += 1;
				}
			}
		}
	}

	function set_srnd() {
		g_srnd[0] = Math.random();
		for (let i=1; i<1001; i++) {
			g_srnd[i] = g_srnd[i-1] + Math.random();
		}
		for (let i=0; i<1000; i++) {
			g_srnd[i] = g_srnd[i]/g_srnd[1000];
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( stateMax, vIndex, v_nElectron )
		evolve:			iterateLDA,				// iterateLDA(stateMax,iterMax)

		setNe:			setNumberOfElectron,	// setNumberOfElectron( ne )
		setCloud:		setCloud,				// setCloud( ist );
		setMixing:		setMixing,				// setMixing( mix )
		setBroadening:	setBroadening,			// setBroadening( broadening )

		getAUinSI:		function() { return [ g_auLength, g_auTime, g_auEnergy, g_au2eV ]; },
		getSysParam:	function() { return [ g_NNx, g_NNy, g_dx ]; },
		getNow:			function() { return [ g_iterCount, totalEnergy() ]; },

		getOccupation:	function(ist) { return g_occupation[ist]; },
		getStEnergy:	function(ist) { return g_sdEnergy[ist]; },
		getState:		function(ist,i,j) { return g_sdState[ist][i][j]; },
		getDensity:		function(i,j) { return g_rho[i][j]; },
		getVext:		function(i,j) { return g_vvext[i][j]; },
		getVeff:		function(i,j) { return g_vv[i][j]; },
		getVh:			function(i,j) { return g_vvh[i][j]; },
		getVx:			function(i,j) { return g_vvx[i][j]; },
		getVc:			function(i,j) { return g_vvc[i][j]; },
		getCloud:		function(i,j) { return g_cloud[i][j]; },
	};

})(); // ====================  sampleLDA2D end  ====================


const js024 = (function(){ // ====================  js Module  ====================

	const theModule = sampleLDA2D;
	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;					// canvas2d
	let ctx;

	let v_stateMax = 10;
	let v_vIndex = 0;			// 0:parabolic, 1:well, 2:well
	let v_nElectron = 4;
	let v_iterMax = 2;

	let p_NNx, p_NNy, p_dx; // = theModule.getSysParam();
	let iterCount, totalEnergy;
	let stateList = [];
  let stateOccupationList = []
  let stateEnergyList = [];

	let dispMode = 3;
	let dispOrbit = 0;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;
	let neChanged = false;
  let breakFlag = false;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();
		viewHome();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			theModule.init( v_stateMax, v_vIndex, v_nElectron ); // ( nn, BoxSizeInNM, contTemp )
			[ p_NNx, p_NNy, p_dx ] = theModule.getSysParam();
			// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
			g3d.init( p_NNx, p_NNy, p_NNy, p_dx, xCanvasSize, yCanvasSize, 300, 20 );

		}

		if ( neChanged ) {
			neChanged = false;
			theModule.setNe(v_nElectron);
		}

		if ( !pauseFlag ) {
			theModule.evolve( v_stateMax, v_iterMax );
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_stateMax, v_iterMax );
			inStepFlag = true;
		}
		setStateList();

		draw( ctx, dispMode );

		requestAnimationFrame(animate);
	}

  function setStateList() {
    for (let i=0; i<v_stateMax; i++) {
      stateList[i] = i;
      stateOccupationList[i] = theModule.getOccupation(i)
      stateEnergyList[i] = theModule.getStEnergy(i);
    }
  }


	// --------------------  draw  --------------------

	const gColor = { orb:"#dddd00", dens:"#dd88dd",
					Vext:"#00dd00", Veff:"#0088ff", Vh:"#4444ff", Vxc:"#8800ff", Vx:"#aa00ff",Vc:"#8888ff" };

	function draw( ctx, dispMode ) {
		const NNx = p_NNx, dx = p_dx;
		const xp = 30, yp = 20, xtabp = 320;

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		if (dispMode==0) { // 1D density(x,0) + Vext,Veff,Vh,Vxc
			draw1D( ctx, 0, xp, yp );
		} else if (dispMode==1) { // 1D orbit(x,0) + Vext
			draw1D( ctx, 1, xp, yp );

		} else if (dispMode==2) { // 2D density(x,y)
			draw2D( ctx, -1, xp, yp);
			ctx.fillText("density(x,y)", xp, yCanvasSize-10);
		} else if (dispMode==3) { // 2D orbit(x,y)
			draw2D( ctx, dispOrbit, xp, yp);
			ctx.fillText("orbit(x,y)", xp, yCanvasSize-10);

		} else if (dispMode==4) { // 3D view - density(x,y)
			const densityFunc = function(i,j) { return 200.0*theModule.getDensity(i,j); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, densityFunc, 0.5, 1 );
			ctx.fillText("density(x,y)", xp, yCanvasSize-10);
		} else if (dispMode==5) { // 3D view - orbit(x,y)
			const orbitFunc = function(i,j) { return 10.0*theModule.getState(dispOrbit,i,j); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, orbitFunc, 0.5, 1 );
			ctx.fillText(`orbit = | ${dispOrbit} >`, xp, yCanvasSize-50);
			ctx.fillText(`orbit energy = ${theModule.getStEnergy(dispOrbit).toFixed(6)}`, xp, yCanvasSize-30);
			ctx.fillText(`orbit occupation = ${theModule.getOccupation(dispOrbit).toFixed(6)}`, xp, yCanvasSize-10);
		} else if (dispMode==6) { // 3D view - cloud(|orbit|^2)
			if ( !pauseFlag || inStepFlag ) theModule.setCloud(dispOrbit);
			inStepFlag = false;
			const cloudFunc = function(i,j) { return 0.5*theModule.getCloud(i,j); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, cloudFunc, 0.5, 1 );
			ctx.fillText("cloud", xp, yCanvasSize-10);
		} else if (dispMode==7) { // 3D view - (density+Vext)(x,y)
			const zFunc = function(i,j) { return 200.0*theModule.getDensity(i,j) + 0.4*theModule.getVext(i,j) };
			const colorFunc = function(i,j) {
				const z = 200.0*theModule.getDensity(i,j);
				const th = (18120 - Math.floor(180.0*z/g3d.cz0))%360;
				return ( Math.abs(z/g3d.cz0)>0.005 ) ? `hsl(${th},100%,50%)` : `hsl(${th},30%,20%)`;
			}
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, colorFunc, 1 );
			ctx.fillText("(density + Vext)(x,y)", xp, yCanvasSize-10);
		} else if (dispMode==8) { // 3D view - Vext(x,y)
			const zFunc = function(i,j) { return 0.4*theModule.getVext(i,j); };
			// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vext, 1 );
			ctx.fillText("Vext(x,y)", xp, yCanvasSize-10);
		} else if (dispMode==9) { // 3D view - Veff(x,y)
			const zFunc = function(i,j) { return 0.4*theModule.getVeff(i,j); };
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Veff, 1 );
			ctx.fillText("Veff(x,y)", xp, yCanvasSize-10);
		} else if (dispMode==10) { // 3D view - Vh(x,y)
			const zFunc = function(i,j) { return 0.4*theModule.getVh(i,j)*10.0; };
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vh, 1 );
			ctx.fillText("Vh(x,y) X 10", xp, yCanvasSize-10);
		} else if (dispMode==11) { // 3D view - Vxc(x,y)
			const zFunc = function(i,j) { return 0.4*(theModule.getVx(i,j)+theModule.getVc(i,j))*10.0; };
			g3d.drawGrid2D( ctx, 0.0, zFunc, gColor.Vxc, 1 );
			ctx.fillText("Vxc(x,y) x 10", xp, yCanvasSize-10);
		}

		if ( dispMode<=3 ) { // orbit energy and occupation table
			ctx.fillStyle = "#888888";
			ctx.fillText("orbit   energy(au)", xtabp, 40);
			for (let ist=0; ist<v_stateMax; ist++) {
				const col = `hsl(${ist*30},100%,50%)`;
				ctx.fillStyle = col;
				ctx.fillText(`|${ist}>   ${(theModule.getStEnergy(ist)).toFixed(6)}`, xtabp, 240-ist*20);
			}
			ctx.fillStyle = "#888888";
			ctx.fillText("orbit  occupation", xtabp, 280);
			for (let ist=0; ist<5; ist++) {
				const occ = theModule.getOccupation(ist);
				ctx.fillStyle = `hsl(${240-120*occ},70%,70%)`;
				ctx.fillText(`|${ist}>   ${(occ).toFixed(6)}`, xtabp, 380-ist*20);
			}
		}

		//
		[ iterCount, totalEnergy ] = theModule.getNow();
		ctx.fillStyle = "#888888";
		ctx.fillText(`box size : ${NNx*dx} (au)`, xtabp, yCanvasSize-50);
		ctx.fillText(`iteration = ${iterCount}`, xtabp, yCanvasSize-30);
		ctx.fillText(`Energy = ${totalEnergy.toFixed(6)}`, xtabp, yCanvasSize-10);
	}

	function draw2D( ctx, ist, xp, yp ) {
		const nnx=p_NNx, nny=p_NNy, pmag = 1000.0, rmag = 12000.0;
		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, 256, 256 );

		for (let i=0; i<nnx; i++) {
			for (let j=0; j<nny; j++) {
				const g = Math.floor(theModule.getVext(i,j)*10);
				let r,b;
				if (ist>=0) { // orbit
					const p = theModule.getState(ist,i,j)*pmag;
					if (p>=0) {
						r = Math.floor(p); if (r>255) r = 255;
						b = 0;
					} else {
						r=0;
						b = Math.floor(-p); if (b>255) b = 255;
					}
				} else { // density
					const p = theModule.getDensity(i,j)*rmag;
					r = Math.min(Math.floor(p),255);
					b = r;
				}
				ctx.fillStyle = `rgb(${r},${g},${b})`;
				ctx.fillRect(i*4+xp,(nny-j-1)*4+yp,4,4);
			}
		}
		ctx.fillStyle = "#888888";
		if (ist>=0) ctx.fillText("orbit - red:ph(x,y)>0 blue:ph(x,y)<0", xp,300);
		if (ist<0) ctx.fillText("density(x,y) - magenta", xp,300);
		ctx.fillText("potential green:Vext(x,y)", xp, 320);
	}

	function draw1D( ctx, mode, xp, yp ) {
		const nnx=p_NNx, nny=p_NNy, nst = v_stateMax;
		const y0 = 280, pmag = 100.0, rmag = 2000.0, emag = 12.0, jc = nny/2, tb = 70;

		ctx.strokeStyle = "#888800";
		ctx.strokeRect( xp, yp, 256, 360 );

		drawLine( ctx, xp-10, y0, xp+266, y0, "#444444" ); // base line
		drawFunc( ctx, function(i) { return theModule.getVext(i,jc)*emag; }, xp, y0, gColor.Vext ); // Vext(x,0)
		if ( mode==0 ) drawFunc( ctx, function(i) { return theModule.getDensity(i,jc)*rmag; }, xp, y0, gColor.dens );
		if ( mode==1 ) {
			for (let ist=nst-1; ist>=0; ist--) { // state(x,0)
				const func = function(i) { return theModule.getState(ist,i,jc)*pmag+theModule.getStEnergy(ist)*emag; };
				drawFunc( ctx, func, xp, y0, `hsl(${ist*30},100%,50%)` )
			}
		} else if ( mode==0 ) {
			drawFunc( ctx, function(i) { return theModule.getVeff(i,jc)*emag; }, xp, y0, gColor.Veff );
			drawFunc( ctx, function(i) { return 10.0*theModule.getVh(i,jc)*emag; }, xp, y0, gColor.Vh );
			drawFunc( ctx, function(i) {
				return 10.0*(theModule.getVx(i,jc)+theModule.getVc(i,jc))*emag; }, xp, y0, gColor.Vxc );
		}
		if ( mode==0 ) {
			ctx.font = "14px 'sans-serif'";
			ctx.fillStyle = gColor.dens; ctx.fillText("density(x,0)", xp, yCanvasSize-50);
			ctx.fillStyle = gColor.Vext; ctx.fillText("Vext(x,0)", xp, yCanvasSize-30);
			ctx.fillStyle = gColor.Veff; ctx.fillText("Veff(x,0)", xp+tb, yCanvasSize-30);
			ctx.fillStyle = gColor.Vh; ctx.fillText("Vh(x,0)", xp+tb*2, yCanvasSize-30);
			ctx.fillStyle = gColor.Vxc; ctx.fillText("Vxc(x,0)", xp+tb*3, yCanvasSize-30);
			ctx.font = "16px 'sans-serif'";
		} else if ( mode==1 ) {
			ctx.fillStyle = "#888888"; ctx.fillText("orbit(x,0)   |0>, |1>, ..., |9>", xp, yCanvasSize-50);
			ctx.fillStyle = gColor.Vext; ctx.fillText("Vext(x,0)", xp, yCanvasSize-30);
		}
		ctx.fillStyle = "#888888"; ctx.fillText("along x-axis", xp, yCanvasSize-10);
	}

	function drawFunc( ctx, func, xp, y0, color ) {
		const nnx=p_NNx, nny=p_NNy;

		ctx.strokeStyle = color;
		ctx.beginPath();
		for (let i=0; i<nnx; i++) {
			ctx.lineTo(i*4+xp,y0-func(i));
		}
		ctx.stroke();
	}

	function drawLine( ctx, x1, y1, x2, y2, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	}


	// --------------------  graphics 3D (field) module  --------------------
	//
	// ver 0.0.1  2018.12.16  last updated on 2023.03.01
	// ver 0.0.2  2023.03.03  last updated on 2023.06.01

	let g_NNx, g_NNy, g_NNz, g_dx, g_dy, g_dz, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift;

	const g3d = {};				// namespace of graphic 3D module

	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.xMax = 0.0;				// x-length of box
	g3d.yMax = 0.0;				// y-length of box
	g3d.zMax = 0.0;				// z-length of box
	g3d.cx0 = 0.0;				// x-component of rotate center
	g3d.cy0 = 0.0;				// y-component of rotate center
	g3d.cz0 = 0.0;				// z-component of rotate center
	g3d.Ax = -Math.PI/15.0;		// rotate angle around x-axis
	g3d.Ay = -Math.PI/15.0;		// rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.xApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.yApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.zApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pxApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pyApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.pzApex = [0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0];
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
					[2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid

	// g3d.init( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift );
	g3d.init = function( NNx, NNy, NNz, dx, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_NNx = NNx; g_NNy = NNy; g_NNz = NNz;
		g_dx = dx; g_dy = dx; g_dz = dx;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.setSize();
	}

	g3d.setSize = function() {
		g3d.xMax = g_NNx*g_dx;		// x-length of box
		g3d.yMax = g_NNy*g_dy;		// y-length of box
		g3d.zMax = g_NNz*g_dz;		// z-length of box
		g3d.cx0 = 0.5*g3d.xMax;		// x-component of rotate center
		g3d.cy0 = 0.5*g3d.yMax;		// y-component of rotate center
		g3d.cz0 = 0.5*g3d.zMax;		// z-component of rotate center
	};

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = g_xCanvasSize/2, yCenter = g_yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_NNx)*g_NNy;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = xBoxSize/(g_NNx*g_dx)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotateXY = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate param
		g3d.rotateApexXY();     // box Apex rotate--> pxApex[i],pyApex[i],pzApex[i]
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g3d.xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g3d.yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g3d.zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g3d.xMax;
		g3d.cy0 = 0.5*g3d.yMax;
		g3d.cz0 = 0.5*g3d.zMax;
	};

	g3d.rotateApexXY = function() { // rotate box apex
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.drawRotatedDisc = function(ctx, x,y,z,r,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1 = cosAy*(x-cx0)+sinAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cx0
		const y1 = cosAx*(y-cy0)-sinAx*(z-cz0) + cy0
		//z1 =-sinAy*(x-cx0)+cosAy*(sinAx*(y-cy0)+cosAx*(z-cz0)) + cz0
		g3d.drawDisc(ctx, x1*sc+xp,y1*sc+yp,r,color);
	};

	g3d.drawRotatedLine = function(ctx, x1,y1,z1,x2,y2,z2,color,sc,xp,yp) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		const x1p = cosAy*(x1-cx0)+sinAy*(sinAx*(y1-cy0)+cosAx*(z1-cz0)) + cx0
		const y1p = cosAx*(y1-cy0)-sinAx*(z1-cz0) + cy0
		const x2p = cosAy*(x2-cx0)+sinAy*(sinAx*(y2-cy0)+cosAx*(z2-cz0)) + cx0
		const y2p = cosAx*(y2-cy0)-sinAx*(z2-cz0) + cy0
		g3d.drawLine(ctx, x1p*sc+xp,y1p*sc+yp,x2p*sc+xp,y2p*sc+yp,color);
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark = 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1=g3d.pxApex[iApex]*sc+xp, y1=g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2=g3d.pxApex[iApex]*sc+xp, y2=g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function(ctx, x, y, r, color) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	// --------------------  end of graphics 3D (field) module  --------------------

	// g3d_extension grid2d  created 2023.06.01, last updated 2023.06.04
	// g3d.drawGrid2D( ctx, rotAngle, zFunc, colorFactor, inc [, showBox] )
	g3d.drawGrid2D = function ( ctx, rotAngle, zFunc, colorFactor, inc, showBox ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc();
		const nnx = g_NNx, nny = g_NNy, threshold = g3d.drawGrid2D.threshold;

		g3d.set3DRotateXY(rotAngle);
		if ( (showBox & 1)>0 || showBox==undefined ) g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let jj=0; jj<nny; jj+=inc) {
			let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
			for (let ii=0; ii<nnx; ii++) {
				let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
				if (i<0 || i+1>nnx-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i+1,nny-j-1);
				const x1 = (i+1)*g_dx, y1 = j*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		for (let ii=0; ii<nnx; ii+=inc) {
			let i=ii;if (g3d.pzApex[1]-g3d.pzApex[0]<0) i=nnx-ii-1;
			for (let jj=0; jj<nny; jj++) {
				let j=jj;if (g3d.pzApex[2]-g3d.pzApex[0]<0) j=nny-jj-1;
				if (j<0 || j+1>nny-1) continue;
				const f = zFunc(i,nny-j-1);
				const x = i*g_dx, y = j*g_dy, z = f + g3d.cz0;
				const f1 = zFunc(i,nny-j-2);
				const x1 = i*g_dx, y1 = (j+1)*g_dy, z1 = f1 + g3d.cz0;
				let colr;
				if ( typeof(colorFactor)=='number' ) {
					const th = (18120 - Math.floor(colorFactor*180.0*(f+f1)/g3d.cz0))%360;
					const a = Math.abs((f+f1)/g3d.cz0);
					colr = (a>threshold) ? `hsl(${th},100%,50%)` : "#444444" ;
				} else if ( typeof(colorFactor)=='string' ) {
					colr = colorFactor;
				} else if ( typeof(colorFactor)=='function' ) {
					colr = colorFactor(i,nny-j-1);
				}
				g3d.drawRotatedLine(ctx, x,y,z,x1,y1,z1,colr,sc,xp,yp);
			}
		}
		if ( (showBox & 2)>0 || showBox==undefined ) g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); //yellow

		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	};
	g3d.drawGrid2D.threshold = 0.005;


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {  // select theme
		v_vIndex = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setNelectron() {  // select theme
		var ne = 0 + document.getElementById("slct_Nelectron").selectedIndex;
		v_nElectron = ne;
		neChanged = true;
	}

	function setDispMode() {  // select dispMode
		const dom = document.getElementById("slct_orbit");
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
		dom.disabled = (dispMode==3 || dispMode==5 || dispMode==6 ) ? false : true;
	}

	function setOrbit() {  // select dispOrbit
		dispOrbit = 0 + document.getElementById("slct_orbit").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(65,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetNelectron( ne ) {
    v_nElectron = ne;
    document.getElementById("slct_Nelectron").selectedIndex = ne;
    neChanged = true;
  }

  function pygetNelectron() {
    return v_nElectron;
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ iterCount, totalEnergy ];
  }

  function pysetDispState( ist ) {
    dispState = ist;
    document.getElementById("slct_dispState").selectedIndex = ist;
  }

  function pygetStateEnergyList() {
    return stateEnergyList;
  }

  function pygetStateList() {
    return [ stateList, stateOccupationList, stateEnergyList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setNelectron:	setNelectron,	// setNelectron()
		setDispMode:	setDispMode,	// setDispMode()
		setOrbit:		setOrbit,		// setOrbit()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetNelectron: pysetNelectron, // pysetNelectron( ne )
    pygetNelectron: pygetNelectron, // pygetNelectron()
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ iterCount, totalEnergy ]
    pysetDispState: pysetDispState, // pysetDispState( ist )
    pygetStateEnergyList: pygetStateEnergyList, // pygetStateEnergyList()
    pygetStateList: pygetStateList, // pygetStateList() : return [ stateList, stateOccupationList, stateEnergyList ]
	};

})(); // ====================  js024 module end  ====================


const js = js024;
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js024] Real Space DFT - Local Density Approxmation 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" WIDTH="480" HEIGHT="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option selected>electrons in parabollic V=0.5 r^2</option>
<option>electrons in well V=0(r&lt;4) =20(else)</option>
<option>electrons in well V=-1(r&lt;4) =0(else)</option>
<option>electrons in ring V=0(4&lt;r&lt;6) =20(else)</option>
</select>
<br>

<label>number of electron:</label>
<select id="slct_Nelectron" onChange="js.setNelectron()">
<option>0</option><option>1</option><option>2</option><option>3</option><option selected>4</option>
<option>5</option><option>6</option><option>7</option><option>8</option>
</select>
    <span style="margin-right: 140px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option>1d density(x,0) + Vext,Veff,Vh,Vxc(x,0)</option><option>1d orbit(x,0) + Vext(x,0)</option>
<option>2d density(x,y) + Vext(x,y)</option><option selected>2d orbit(x,y) + Vext(x,y)</option>
<option>grid2d: density(x,y)</option><option>grid2d: orbit(x,y)</option>
<option>grid2d: cloud(|orbit|^2)</option><option>grid2d: (density+Vext)(x,y)</option>
<option>grid2d: Vext(x,y)</option><option>grid2d: Veff(x,y)</option>
<option>grid2d: Vh(x,y) x 10</option><option>grid2d: Vxc(x,y) x 10</option>
</select>
    <span style="margin-right: 20px;"></span>
<label>orbit:</label>
<select id="slct_orbit" onChange="js.setOrbit()">
<option selected>0</option><option>1</option><option>2</option><option>3</option><option>4</option>
<option>5</option><option>6</option><option>7</option><option>8</option><option>9</option>
</select>
<br>

<button onClick="js.viewHome()">return to initial view</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# exec html-js code, and python control

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
time.sleep(3)

#reset
eval_js( 'js.reset()' )
print("-- reset --")

# python control
for i in range(10):
  [ iterCount, totalEnergy ] = eval_js( 'js.pygetData({})'.format(i) )
  print(
    "i=",i,
    ",\t iter count =", '{:.0f}'.format(iterCount),
    ", total energy =", '{:.8g}'.format(totalEnergy), "(au)" )
  time.sleep(2)
#

# print state energy list
[ stateList, stateOccupationList, stateEnergyList ] = eval_js( 'js.pygetStateList()' )
g6List = [ float( '{:.6g}'.format(num) ) for num in stateEnergyList ]
print("")
print( "number of electron =", eval_js('js.pygetNelectron()') )
print( "state list :", stateList )
print( "state occupation list :", stateOccupationList )
print( "state energy list :", g6List )

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# exec html-js code, and python control, change number of electron

import time
import numpy as np
import matplotlib.pyplot as plt


# exec html-js code
exec_html_js()
time.sleep(3)

# reset
eval_js( 'js.reset()' )
print("-- reset --")

# change dispMode
dispMode = 7 #' grid2d: (density+Vext)(x,y)'
eval_js('js.pysetDispMode({})'.format(dispMode) )
print("-- set dispmode = 7 - grid2d:(density+Vext)(x,y)' -- ")

# python control
for i in [ 1, 2, 3, 4 ]:
  eval_js( 'js.pysetNelectron({})'.format(i) )
  print("-- set number of electron =", i, " --" )
  time.sleep(5)
  # print state energy list
  [ stateList, stateOccupationList, stateEnergyList ] = eval_js( 'js.pygetStateList()' )
  g6List = [ float( '{:.6g}'.format(num) ) for num in stateEnergyList ]
  print( "\t number of electron =", eval_js('js.pygetNelectron()') )
  print( "\t state list :", stateList )
  print( "\t state occupation list :", stateOccupationList )
  print( "\t state energy list :", g6List )
  print("")
#

time.sleep(1)
# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")